###**CHART** **PIPELINE**

Montaggio del drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installazione

In [2]:
!pip install numpy
!pip install torch
!pip install cython
!pip install opencv-python
!pip install h5py
!pip install pytesseract
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pycocotools
!pip install easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,646 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

Import

In [3]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
import cv2
from google.colab.patches import cv2_imshow
import pytesseract
import pandas as pd
import os
import re
import numpy as np
import pickle
import shutil

Inizializzazioni delle variabili

In [4]:
figCap_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/PDFigCapX/code/FigCap.py"
input_file_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/input_file"
output_file_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/output_file/"
inputChartOCR_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/inputChartOCR"
outputChartOCR_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/outputChartOCR"
output_final_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/output_final"


Definizione delle funzioni

In [5]:
def load_object_from_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)



Run di CapFigX

In [6]:
!python {figCap_path} \
--input_path {input_file_path} \
--output_path {output_file_path}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray rad

Copia output CapFigX nell'input di chartOcr

In [7]:
import os
estensioni_immagini=['.jpg']
if os.path.exists(output_file_path):
        lista_elementi = os.listdir(output_file_path)
        cartelle = [elemento for elemento in lista_elementi if os.path.isdir(os.path.join(output_file_path, elemento))    ]
else:
        print("La cartella {} non esiste.".format(output_file_path))
        #exit(1)
cartelle= [c for c in cartelle if c!="xpdf"]
dizionario_pdf_img=dict()
for c in cartelle :
  lista_immagini = [nome for nome in os.listdir(os.path.join(output_file_path,c)) if os.path.splitext(nome)[1].lower() in estensioni_immagini ]
  if len(lista_immagini)>0:
    dizionario_pdf_img[c]=lista_immagini

for d in dizionario_pdf_img.keys():
  percorso_cartella_pdf=os.path.join(inputChartOCR_path,d)
  os.makedirs(percorso_cartella_pdf, exist_ok=True)
  for im in dizionario_pdf_img[d]:
    shutil.copy(  os.path.join(output_file_path,d,im)  , percorso_cartella_pdf)

Run di ChartOCR

In [8]:
os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule/models/py_utils/_cpools")
!python setup.py build_ext --inplace

running build_ext
building 'top_pool' extension
Emitting ninja build file /content/drive/.shortcut-targets-by-id/1bQFNhEP0_9OfhS476MaXFYzpHMRqKXpw/ASDP Chart Extraction/DeepRule/models/py_utils/_cpools/build/temp.linux-x86_64-cpython-310/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/1] c++ -MMD -MF '/content/drive/.shortcut-targets-by-id/1bQFNhEP0_9OfhS476MaXFYzpHMRqKXpw/ASDP Chart Extraction/DeepRule/models/py_utils/_cpools/build/temp.linux-x86_64-cpython-310/src/top_pool.o'.d -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/torch/include -I/usr/local/lib/python3.10/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.10/dist-packages/torch/include/TH -I/usr/local/lib/python3.10/dist-packages/torch/include/THC -I/usr/inc

In [9]:
os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule/external")
!make

python setup.py build_ext --inplace
nms.c: In function ‘__pyx_pf_3nms_2soft_nms’:
nms.c:6295:32: warning: comparison of integer expressions of different signedness: ‘int’ and ‘unsigned int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wsign-compare-Wsign-compare]8;;]
 6295 |       __pyx_t_9 = (__pyx_v_pos < __pyx_v_N);
      |                                ^
nms.c:6806:32: warning: comparison of integer expressions of different signedness: ‘int’ and ‘unsigned int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wsign-compare-Wsign-compare]8;;]
 6806 |       __pyx_t_9 = (__pyx_v_pos < __pyx_v_N);
      |                                ^
nms.c: In function ‘__pyx_pf_3nms_4soft_nms_merge’:
nms.c:8093:32: warning: comparison of integer expressions of different signedness: ‘int’ and ‘unsigned int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wsign-compare-Wsign-compare]8;;]
 8093 |       __pyx_t_9 = (__pyx_v_pos < _

In [10]:
os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule")

!python "test_pipeline_new.py" --image_dir {inputChartOCR_path} --output_dir {outputChartOCR_path}

loading parameters at iteration: 50000
loading from cache file: data/clsdata(1031)/cache/cls_val2019.pkl
loading annotations into memory...
Done (t=1.88s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
building neural network...
module_file: models.CornerNetCls
/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
use kp
total parameters: 199649452
loading parameters...
loading model from data/clsdata(1031)/cache/nnet/CornerNetCls/CornerNetCls_50000.pkl
loading parameters at iteration: 50000
loading from cache file: data/bardata(1031)/cache/chart_val2019.pkl
loading annotations into memory...
Done (t=1.25s)
creating index...
index created!
building neural network...
module_file: models.CornerNetPureBar
use kp pure
total parameters: 198592652
loading parameters...
loading 

Salvataggio sottoforma di file excel dell'output di chartOCR

In [12]:
diz_df = dict()

if os.path.exists(outputChartOCR_path):
    lista_nomi_pdf = os.listdir(outputChartOCR_path)
    for nome_pdf in lista_nomi_pdf:
      diz_df[nome_pdf] =[]
      lista_nomi_img=os.listdir(os.path.join(outputChartOCR_path,nome_pdf))
      for nome_img in lista_nomi_img:
        if os.path.exists(os.path.join(outputChartOCR_path,nome_pdf,nome_img,"result.jpg")):
          labelled_img = os.path.join(outputChartOCR_path,nome_pdf,nome_img,"result.jpg")
          print("SI RESULT")

        if os.path.exists(os.path.join(outputChartOCR_path,nome_pdf,nome_img,"chart_object.pkl")):
          pkl_chart = os.path.join(outputChartOCR_path,nome_pdf,nome_img,"chart_object.pkl")
          chart = load_object_from_file(pkl_chart)
          print("SI PKL")
          #creiamo dizionaro di df cosi li salvo su excel dopo
          diz_df[nome_pdf].append(chart.df_table)

        else:
          print("NO PKL")


for k in diz_df.keys():
  cartella_pdf=os.path.join(output_final_path,k)
  if not os.path.exists(cartella_pdf):
        os.makedirs(cartella_pdf)

  for i,df in enumerate(diz_df[k]):
    if df is not None:
      df.to_excel(os.path.join(cartella_pdf,"dataframes.xlsx"), sheet_name=f'img_{i+1}')

SI RESULT
NO PKL
SI RESULT
NO PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
NO PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
NO PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
SI RESULT
SI PKL
NO PKL
